In [1]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
builder = (SparkSession.builder
           .appName("config-checkpoints")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3b50e98-b482-40d0-a8e3-0f9724998c5c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in centra

In [3]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

In [4]:
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load())

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('country', StringType(), True)])

df = df.withColumn('value', from_json(col('value').cast("STRING"), schema))

In [6]:
from pyspark.sql.functions import col

df = df.select(
    col('value.id').alias('id'),
    col('value.name').alias('name'),
    col('value.age').alias('age'),
    col('value.gender').alias('gender'),
    col('value.country').alias('country'))

In [7]:
query = (df.writeStream
   .format("console")
   .outputMode("append")
   .option("checkpointLocation", "/opt/workspace/data/checkpoint")
   .start())

-------------------------------------------
Batch: 0
-------------------------------------------
+---+-------+---+------+---------+
| id|   name|age|gender|  country|
+---+-------+---+------+---------+
|100| user43| 65|     F|Australia|
| 61| user68| 27|     M|Australia|
| 35| user54| 37|     M|   Brazil|
|  9| user30| 18|     M|    China|
|  9| user82| 24|     M|   Canada|
| 22| user40| 19|     F|   Brazil|
| 78| user81| 43|     F|Australia|
|  4| user90| 60|     F|    China|
| 16| user31| 54|     F|Australia|
| 36|user100| 48|     M|Australia|
| 20| user14| 23|     F|    China|
| 82| user21| 60|     M|      USA|
| 90| user39| 50|     M|Australia|
| 52|  user6| 51|     M|       UK|
| 86| user61| 65|     F|       UK|
| 46| user38| 46|     F|    China|
|  3| user54| 45|     F|   Canada|
| 32| user24| 49|     M|       UK|
| 29|  user2| 59|     M|    India|
| 57| user72| 46|     F|   Brazil|
+---+-------+---+------+---------+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+---+------+---+------+-------+
| id|  name|age|gender|country|
+---+------+---+------+-------+
| 40|user61| 28|     F|     UK|
+---+------+---+------+-------+



-------------------------------------------
Batch: 2
-------------------------------------------
+---+------+---+------+-------+
| id|  name|age|gender|country|
+---+------+---+------+-------+
| 60|user12| 37|     F|     UK|
+---+------+---+------+-------+



In [12]:
query.stop()

In [11]:
query = (df.writeStream
   .format("console")
   .outputMode("append")
   .option("checkpointLocation", "/opt/workspace/data/checkpoint")
   .start())

-------------------------------------------
Batch: 3
-------------------------------------------
+---+------+---+------+---------+
| id|  name|age|gender|  country|
+---+------+---+------+---------+
| 63|user27| 26|     M|Australia|
| 23|user92| 35|     F|    China|
| 11|user57| 59|     F|   Canada|
| 17|user32| 56|     F|       UK|
+---+------+---+------+---------+



-------------------------------------------
Batch: 4
-------------------------------------------
+---+------+---+------+---------+
| id|  name|age|gender|  country|
+---+------+---+------+---------+
|  2|user22| 53|     M|Australia|
+---+------+---+------+---------+



In [ ]:
query.stop()

In [9]:
spark.stop() 